In [ ]:
import pyrho
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
def get_trig_function(A, B, m, n):
    """Return a trigonometric function of x and y."""
    def trig_function(x, y):
        return A * np.sin(m * x * 2 * np.pi) * np.sin(n * y * 2 * np.pi) + B # V = A * n**2 + A * m**2 + B
    return trig_function
    
f1 = get_trig_function(1, 0.1, 3, 1)

In [ ]:
xx = np.linspace(0, 1, 512)
yy = np.linspace(0, 1, 512)
XX, YY = np.meshgrid(xx, yy, indexing="ij")
plt.imshow(f1(XX, YY))


In [ ]:
from pyrho.pgrid import PGrid
pgrid = PGrid(grid_data=f1(XX, YY), lattice=[[1, 0], [0, 1]])

In [ ]:
pg2 = pgrid.get_transformed(sc_mat=[[2, 0], [0, 2]], origin=(0, 0), grid_out=(512, 512))

In [ ]:
# get randomly oriented square lattice
def get_random_square_lattice(side_length, uc_lattice):
    """Return a random square lattice."""
    theta = np.random.uniform(0, 2 * np.pi)
    cube_latt = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]]) * side_length
    # return the cube lattice in terms of the unit cell lattice
    return np.dot(cube_latt, np.linalg.inv(uc_lattice))



In [ ]:
res = get_random_square_lattice(1.5, np.eye(2))
pg2 = pgrid.get_transformed(sc_mat=res, origin=(0, 0), grid_out=(512, 512))
plt.imshow(pg2.grid_data)

In [ ]:
for i in range(5):
    res = get_random_square_lattice(1.5, np.eye(2))
    pg2 = pgrid.get_transformed(sc_mat=res, origin=(0, 0), grid_out=(512, 512))
    res = get_random_square_lattice(2.0, np.eye(2))
    pg2 = pgrid.get_transformed(sc_mat=res, origin=(0, 0), grid_out=(512, 512))
    fres = np.abs(np.fft.fft2(pg2.grid_data))
    fres = np.fft.fftshift(fres)
    width = 10
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 7))
    ax1.imshow(fres[256-width:256+width, 256-width:256+width], vmin=0, vmax=3E4)
    ax2.imshow(pg2.grid_data)